## A Simple RAG Chain
This notebook contains experiments for building a simple RAG chain which can answer user's questions regarding
[Blueprint for an AI Bill of Rights: Making Automated Systems Work for the American People](https://www.whitehouse.gov/wp-content/uploads/2022/10/Blueprint-for-an-AI-Bill-of-Rights.pdf) and [National Institute of Standards and Technology (NIST) Artificial Intelligent Risk Management Framework](https://nvlpubs.nist.gov/nistpubs/ai/NIST.AI.600-1.pdf)

In [3]:
# imports
import tiktoken
import os
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from qdrant_client.http.models import Distance, VectorParams
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import Qdrant
from langchain_openai.llms import OpenAI
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.conversation.memory import ConversationSummaryBufferMemory
from langchain.chains.conversation.base import ConversationChain
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
from langchain_core.prompts import (ChatMessagePromptTemplate, SystemMessagePromptTemplate, 
                                    AIMessagePromptTemplate, HumanMessagePromptTemplate)
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts.chat import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.output_parsers import OutputFixingParser

In [5]:
from dotenv import load_dotenv; _ = load_dotenv()

In [3]:
import nest_asyncio; nest_asyncio.apply()

## A Simple RAG Chain

In [11]:
prompt = """
Please answer the question below using the provided context. If the question cannnot be answered
using the context, politely state that you can't answer that question.

Question:
{question}

Context:
{context}
"""
prompt = ChatPromptTemplate.from_template(prompt)

In [12]:
embedding = OpenAIEmbeddings(model='text-embedding-3-small')
retriever = QdrantVectorStore.from_existing_collection(
    collection_name='ai_ethics_te3_small',
    embedding=embedding,
    url=os.environ.get('QDRANT_DB'),
    api_key=os.environ.get('QDRANT_API_KEY')
).as_retriever()

In [13]:
llm = ChatOpenAI(model='gpt-4o', temperature=0)

In [14]:
rag_chain = ({'context': retriever, 'question': RunnablePassthrough()}
             | prompt
             | llm)

In [15]:
async def message(message):
    async for r in rag_chain.astream(message):
        print(r.content, end="")

In [16]:
await message("What is the AI Bill of rights?")

The AI Bill of Rights, as described in the provided context, is a framework published by the White House Office of Science and Technology Policy. It aims to support the development of policies and practices that protect civil rights and promote democratic values in the building, deployment, and governance of automated systems. The document is non-binding and does not constitute U.S. government policy. It outlines principles to guide the use of automated systems, particularly those that have the potential to impact individuals' or communities' rights, opportunities, or access to critical resources or services.

If you need more specific details or further elaboration on the principles and practices included in the AI Bill of Rights, please let me know!

In [17]:
await message("How can we make AI safer for all humanity?")

The context provided outlines several principles and frameworks aimed at making AI systems safer and more effective. Here are some key strategies mentioned:

1. **Adherence to Principles**: Federal agencies are required to follow principles such as ensuring AI is lawful, purposeful, accurate, reliable, safe, secure, understandable, responsible, traceable, regularly monitored, transparent, and accountable.

2. **Regulatory Frameworks**: The National Highway Traffic Safety Administration (NHTSA) and other agencies have rigorous standards and independent evaluations to ensure safety without stifling innovation.

3. **Risk Management**: Organizations are using innovative solutions like risk assessments, auditing mechanisms, and ongoing monitoring to mitigate risks associated with AI.

4. **Stakeholder Engagement**: Expanding opportunities for meaningful stakeholder engagement in the design of AI programs and services.

5. **Research and Development**: The National Science Foundation (NSF) 

## Adding conversation history
The chain above doesn't keep track of the conversation history between the user and the bot. We also define a more 
complicated chain below which keeps track of the user's conversation memory and additionally attempts to ensure that
the retrieved context is helpful in answering any questions from the user. 

The downside is that this uses **ConversationChain** which doesn't support async output streaming (Support for RunnableWithMessageHistory seems lacking so far).

In [33]:
bill_of_rights = PyMuPDFLoader('Blueprint-for-an-AI-Bill-of-Rights.pdf').load()
nist = PyMuPDFLoader('NIST.AI.600-1.pdf').load()

In [87]:
def tiktoken_len(text, model='gpt-4o'):
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

In [88]:
llm = ChatOpenAI(model='gpt-4o', temperature=0)
summary_chain = load_summarize_chain(llm=llm)

In [89]:
bill_of_rights_text = "\n\n\n".join([d.page_content for d in bill_of_rights])
nist_text = "\n\n\n".join([d.page_content for d in nist])

Let's look at the length of the combined documents

In [90]:
tiktoken_len(bill_of_rights_text)

44747

In [91]:
tiktoken_len(nist_text)

35902

These documents are small enough to fit in the model's context window. So let's quickly summarize them both

In [92]:
resp = summary_chain.invoke(bill_of_rights)

In [93]:
bill_of_rights_summary = resp['output_text']

In [94]:
resp = summary_chain.invoke(nist)

In [95]:
nist_summary = resp['output_text']

In [96]:
RAG_PROMPT = """
Please follow the following instructions: 
1. You are a helpful AI assistant who can help answer a user's questions regarding the documents AI Bill of Rights and
NIST AI Risk Assessment Framework. 
2. A summary of both the documents is provided below. A user will occassionally ask
you questions about these documents along with optional context to help you answer these questions. 
3. Please answer the user's questions based on the context they provide.
4. Only if the user provides no context, rely on the summaries below. 
5. Let the user know you cannot answer the question if it is not based on their provided context or the summaries
below.

AI Bill of Rights Summary
{bill_of_rights_summary}

NIST AI Risk Assessment Framework Summary
{nist_summary}
"""

rag_system_prompt = (ChatPromptTemplate.from_template(RAG_PROMPT)
              .partial(bill_of_rights_summary=bill_of_rights_summary)
              .partial(nist_summary=nist_summary))

In [15]:
import pickle
with open('rag_system_prompt.pkl', 'wb') as f:
    pickle.dump(rag_system_prompt, f)

In [4]:
import pickle
with open('rag_system_prompt.pkl', 'rb') as f:
    rag_system_prompt = pickle.load(f)

In [3]:
embedding = OpenAIEmbeddings(model='text-embedding-3-small')
retriever = QdrantVectorStore.from_existing_collection(
    collection_name='ai_ethics_te3_small',
    embedding=embedding,
    url=os.environ.get('QDRANT_DB'),
    api_key=os.environ.get('QDRANT_API_KEY')
).as_retriever()

In [5]:
llm = ChatOpenAI(model='gpt-4o', temperature=0)
r = llm.invoke(rag_system_prompt.format())

## A Simple Context Rewriting Chain
Since we're building a conversational bot with some memory, it helps if the RAG chain doesn't return unhelpful context for the bot to answer. We'll define a simple chain component which inspects the question and the context and judges whether the context is actually helpful in answering the question. Otherwise it will return an empty context so the model can rely on summaries and its conversation history

In [6]:
class ContextOutput(BaseModel):
    question: str = Field(description='The question from the user') 
    context: str = Field(description='The context for the question')

output_parser = PydanticOutputParser(pydantic_object=ContextOutput)
instructions = """
1. You are to judge the question and context you see below and assess whether the context provided is actually helpful
in answering the question. 
2. If the context is helpful in answering the question, please return the question and context 
without changes. 
3. Otherwise, please return the question without changes but an empty string as the context.
4. Please follow the formatting instructions below as well

Question:
{question}

Context:
{context}

{format_instructions}
"""
prompt = ChatPromptTemplate.from_messages(
    ('system', instructions)
).partial(format_instructions=output_parser.get_format_instructions())

#output_fixer = OutputFixingParser.from_llm(parser=output_parser, llm=ChatOpenAI())
context_llm = ChatOpenAI(model='gpt-4o', temperature=0)
context_chain = prompt | context_llm |  output_parser

In [7]:
rag_prompt = ChatPromptTemplate.from_template("""
Question:
{question}

Context:
{context}
""")

In [8]:
rag_chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | context_chain
    | {'context': lambda x: x.context, 'question': lambda x: x.question}
    | rag_prompt
    | llm
)

In [9]:
def message(message):
    r = rag_chain.invoke(message)
    return r.content

In [79]:
print(message("What is the AI Bill of Rights?"))

The AI Bill of Rights, formally known as the "Blueprint for an AI Bill of Rights," is a set of guidelines and principles proposed by the White House Office of Science and Technology Policy (OSTP) in the United States. Released in October 2022, it aims to protect the public from potential harms associated with artificial intelligence (AI) and automated systems. The document outlines five key principles designed to ensure that AI technologies are developed and used in ways that are ethical, fair, and respect individual rights.

The five principles are:

1. **Safe and Effective Systems**: AI systems should be developed and deployed in a manner that ensures they are safe and effective. This includes rigorous testing and monitoring to prevent harm and ensure that the systems function as intended.

2. **Algorithmic Discrimination Protections**: AI systems should be designed and used in ways that prevent discrimination and bias. This involves implementing measures to ensure that AI does not p

In [80]:
print(message("What are some of the provisions for data privacy?"))

Data privacy provisions are measures and regulations designed to protect personal data from unauthorized access, use, disclosure, disruption, modification, or destruction. These provisions can be found in various laws, regulations, and best practices globally. Here are some key provisions for data privacy:

1. **Consent**: Organizations must obtain explicit consent from individuals before collecting, using, or sharing their personal data. This consent must be informed, meaning individuals should understand what data is being collected and how it will be used.

2. **Data Minimization**: Only the data necessary for a specific purpose should be collected and processed. This principle helps reduce the risk of unnecessary data exposure.

3. **Purpose Limitation**: Personal data should only be collected for specified, explicit, and legitimate purposes and not further processed in a manner that is incompatible with those purposes.

4. **Data Accuracy**: Organizations must ensure that personal

In [82]:
print(message("Why should individuals worry about AI?"))

The rise of artificial intelligence (AI) has brought about significant advancements and opportunities, but it also raises several concerns that individuals should be aware of. Here are some reasons why people might worry about AI:

1. **Job Displacement**: AI and automation have the potential to replace a wide range of jobs, from manufacturing to white-collar positions. This could lead to significant unemployment and economic disruption if new job opportunities are not created at a similar pace.

2. **Privacy Concerns**: AI systems often rely on large amounts of data to function effectively. This can lead to concerns about how personal data is collected, stored, and used, potentially infringing on individual privacy.

3. **Bias and Discrimination**: AI systems can inadvertently perpetuate or even exacerbate existing biases present in the data they are trained on. This can lead to unfair treatment in areas such as hiring, lending, and law enforcement.

4. **Security Risks**: AI can be u